In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices("GPU"))

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

DATASET_PATH = r"C:\Users\seniv\Downloads\Forensic\balanced_dataset"

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 8

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.20
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)


train_data = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)


val_data = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)


test_data = test_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print("Classes:", train_data.class_indices)

In [ ]:
import os
print("Train classes:", train_data.class_indices)
print("Samples per class (train):")
from collections import Counter
print(Counter(train_data.classes))
print("Validation samples:", len(val_data.filenames))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D

model = Sequential([

    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(256, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    GlobalAveragePooling2D(),

    Dense(256, activation='relu'),
    Dropout(0.4),

    Dense(128, activation='relu'),
    Dropout(0.3),

    Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(1e-4)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
from sklearn.utils import class_weight
import numpy as np

y = train_data.classes  

class_weights_array = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)

class_weights = dict(enumerate(class_weights_array))

print("Class Weights:", class_weights)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(
    "deepfake.keras",      
    monitor="val_loss",            
    save_best_only=True,           
    mode="min",                    
    verbose=1
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,                    
    restore_best_weights=True,     
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    patience=2,                    
    factor=0.5,                    
    min_lr=1e-7,                  
    verbose=1
)

callbacks = [checkpoint, early_stop, reduce_lr]

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=40,
    callbacks=callbacks
)

In [ ]:
model.save("deepfake.keras")

In [ ]:
val_loss, val_acc = model.evaluate(val_data)
print("Validation Accuracy:", val_acc)
print("Validation Loss:", val_loss)